In [1]:
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import torch

image_size=128               #Image_size
filter_size=(3,3)            #Filter_size
num_classes=2                #Number of Classes
stride=1                     #Stride
padding='same'               #Padding
learning_rate = 0.0001        #Learning Rate
number_of_epochs = 20        # Number of Epochs
batch_size = 4               # Batch Size
train_data_dir='/content/drive/MyDrive/IIIT/Projects/Brain_Hemorrage_Dataset/Classification/train'
val_data_dir='/content/drive/MyDrive/IIIT/Projects/Brain_Hemorrage_Dataset/Classification/valid'


#Data Loadder for calculating the mean and std_div for the images in the training dataset
mean_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])
mean_dataset = datasets.ImageFolder(root=train_data_dir,transform=mean_transform)
mean_loader = DataLoader(mean_dataset, batch_size=batch_size, shuffle=True,num_workers=0, pin_memory=True)
imgs = [item[0] for item in mean_loader]
imgs = torch.stack(imgs, dim=0).numpy()
imgs=np.squeeze(imgs)


# Calculate mean over each channel (r,g,b)
mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()
mean=(mean_r,mean_g,mean_b)
# Calculate std over each channel (r,g,b)
std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()
std_div=(std_r,std_g,std_b)


# Transformations to the Training Set
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=(45, 60)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mean,
        std=std_div
    )
])
# Training Dataset
train_dataset = datasets.ImageFolder(root=train_data_dir,transform=train_transform)
# Training Data Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=0, pin_memory=True)


#Transformations to the Validation Set
valid_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mean,
        std=std_div
    )
])
# Validation Dataset
valid_dataset = datasets.ImageFolder(root=val_data_dir,transform=valid_transform)
# Validation Data loaders
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False,num_workers=0, pin_memory=True)

In [ ]:
class CNN(nn.Module):
    def __init__ (self,image,filter_size,num_classes):
        super(CNN,self).__init__()
        self.Conv1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=filter_size,stride=stride,padding=padding)
        self.Conv2=nn.Conv2d(in_channels=32,out_channels=128,kernel_size=filter_size,stride=stride,padding=padding)
        self.Conv3=nn.Conv2d(in_channels=128,out_channels=64,kernel_size=filter_size,stride=stride,padding=padding)
        self.Conv4=nn.Conv2d(in_channels=64,out_channels=32,kernel_size=filter_size,stride=stride,padding=padding)
        self.linear=nn.Linear(104857,num_classes)
        self.BN1=nn.BatchNorm2d(32)
        self.BN2=nn.BatchNorm2d(128)
        self.BN3=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
    def forward(self,x):
        out=self.Conv1(x)
        out=self.BN1(out)
        out=self.relu(out)
        out=self.Conv2(out)
        out=self.BN2(out)
        out=self.relu(out)
        out=self.Conv3(out)
        out=self.BN3(out)
        out=self.relu(out)
        out=torch.flatten(out,start_dim=1)
        out=self.linear(out)
        return out
model=CNN(image_size,filter_size,num_classes)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
print(model)

CNN(
  (Conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (Conv2): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (Conv3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (Conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (linear): Linear(in_features=1048576, out_features=2, bias=True)
  (BN1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)


In [ ]:
for epoch in range(number_of_epochs):
    train_total_loss=0
    valid_total_loss=0
    n_train_samples=0
    n_train_correct=0
    n_test_samples=0
    n_test_correct=0
    model.train()
    for i,(images,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
    model.eval()
    for i,(images,labels) in enumerate(valid_loader):
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        valid_loss=criterion(outputs,labels)
        valid_total_loss=valid_total_loss+valid_loss.item()
        n_test_samples+=labels.size(0)
        n_test_correct += (predicted == labels).sum().item()
    for i,(images,labels) in enumerate(train_loader):
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        train_loss=criterion(outputs,labels)
        train_total_loss=train_total_loss+train_loss.item()
        n_train_samples+=labels.size(0)
        n_train_correct += (predicted == labels).sum().item()
    print('Epoch:',epoch,'  || Training Loss:',train_total_loss,'  || Validation Loss:',valid_total_loss)
    train_acc = 100.0 * n_train_correct / n_train_samples
    valid_acc = 100.0 * n_test_correct / n_test_samples
    print(f'Training Accuracy: {train_acc} %')
    print(f'Validation Accuracy: {valid_acc} %')

Epoch: 0   || Training Loss: 314.8158050004856   || Validation Loss: 67.42227172851562
Training Accuracy: 75.0 %
Validation Accuracy: 80.0 %
Epoch: 1   || Training Loss: 18.986287709970377   || Validation Loss: 19.29910659790039
Training Accuracy: 96.66666666666667 %
Validation Accuracy: 66.66666666666667 %
Epoch: 2   || Training Loss: 12.95381068862946   || Validation Loss: 46.95847463607788
Training Accuracy: 96.66666666666667 %
Validation Accuracy: 66.66666666666667 %
Epoch: 3   || Training Loss: 14.499522459365693   || Validation Loss: 16.56641022861004
Training Accuracy: 96.11111111111111 %
Validation Accuracy: 73.33333333333333 %
Epoch: 4   || Training Loss: 6.156472488944928   || Validation Loss: 32.464298248291016
Training Accuracy: 97.77777777777777 %
Validation Accuracy: 66.66666666666667 %
Epoch: 5   || Training Loss: 5.538781460640134   || Validation Loss: 42.41081237792969
Training Accuracy: 98.33333333333333 %
Validation Accuracy: 60.0 %
Epoch: 6   || Training Loss: 0.129